In [1]:
# Flask、pyngrok、GoogleのAIライブラリをインストールします
!pip install flask pyngrok google-generativeai

In [2]:
import os
from google.colab import userdata

# Google APIキーとngrokのトークンを設定
# 左側の鍵アイコン(Secrets)から設定することもできますが、
# ここでは直接入力する方法を紹介します。
GOOGLE_API_KEY = "あなたのGoogle APIキーを入力"  # 例: "AIzaSy..."
NGROK_AUTHTOKEN = "あなたのNGROKトークンを入力" # 例: "2abcDEfghI..."

# 環境変数に設定
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [3]:
# Flask、pyngrok、GoogleのAIライブラリに加えて、Flask-Corsもインストールします
!pip install flask pyngrok google-generativeai flask-cors

In [4]:
# 以下のコードをColabのサーバー用セルに貼り付けてください

from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_cors import CORS
import google.generativeai as genai
import os

# --- 初期設定 ---
if 'GOOGLE_API_KEY' not in os.environ:
    print("エラー: GOOGLE_API_KEYが設定されていません。")
if 'NGROK_AUTHTOKEN' not in os.environ:
    # もしエラーが出る場合は、ここに直接ngrokトークンを貼り付けてください
    NGROK_AUTHTOKEN = "あなたのNGROKトークンを入力"  #ここに入力!!!!!
else:
    NGROK_AUTHTOKEN = os.environ['NGROK_AUTHTOKEN']

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
app = Flask(__name__)
CORS(app)
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# --- AIモデルの準備 ---
model = genai.GenerativeModel('gemini-1.5-flash')

# --- Webサーバーの処理 ---

# 1. 「タスク提案」用API
@app.route('/api/recommend', methods=['POST'])
def get_recommendation():
    data = request.get_json()
    if not data or 'prompt' not in data: return jsonify({"error": "プロンプトがありません"}), 400
    try:
        response = model.generate_content(data['prompt'])
        return jsonify({"recommendation": response.text})
    except Exception as e: return jsonify({"error": f"AIエラー: {e}"}), 500

# 2. 「文章要約」用API
@app.route('/api/summarize', methods=['POST'])
def summarize_text():
    data = request.get_json()
    if not data or 'text' not in data: return jsonify({"error": "要約するテキストがありません"}), 400
    prompt = f"あなたは優秀な編集者です。以下の文章を、最も重要なポイントがわかるように3つの箇条書きで簡潔に要約してください。\n\n---\n{data['text']}"
    try:
        response = model.generate_content(prompt)
        return jsonify({"summary": response.text})
    except Exception as e: return jsonify({"error": f"AIエラー: {e}"}), 500

# 3. 「レポート構成案」用API
@app.route('/api/create_outline', methods=['POST'])
def create_report_outline():
    data = request.get_json()
    if not data or not all(k in data for k in ['theme', 'keywords', 'word_count']):
        return jsonify({"error": "必要な情報（テーマ、キーワード、文字数）が不足しています"}), 400
    prompt = f"""
    あなたは優秀な学術アドバイザーです。以下の条件でレポートを作成します。レポートの骨子となる構成案を、「序論」「本論1」「本論2」「本論3」「結論」の5つの章立てで提案してください。各章でどのような内容を記述すべきか、具体的な指針も示してください。
    # レポートの条件
    - テーマ: {data['theme']}
    - 必ず含めるべきキーワード: {data['keywords']}
    - 想定文字数: {data['word_count']}字
    """
    try:
        response = model.generate_content(prompt)
        return jsonify({"outline": response.text})
    except Exception as e: return jsonify({"error": f"AIエラー: {e}"}), 500

# 4. 【新機能】「用語解説」用API
@app.route('/api/explain_term', methods=['POST'])
def explain_term():
    data = request.get_json()
    if not data or 'term' not in data:
        return jsonify({"error": "解説する単語がありません"}), 400

    term = data['term']
    context = data.get('context', '指定なし') # contextがなくても動くようにする

    prompt = f"""
    あなたは非常に優秀な家庭教師です。
    以下の単語について、専門知識がない学生にも理解できるように、丁寧かつ詳細に解説してください。

    # 解説する単語
    {term}

    # 文脈（どのような状況でこの単語が出てきたか）
    {context}

    # 解説に含めるべき項目
    1.  **一言での簡単な説明**: まず、この単語が何であるかを一行で要約してください。
    2.  **詳細な解説**: 具体的な例え話を交えながら、その仕組みや背景を詳しく説明してください。
    3.  **重要性**: なぜこの単語が重要なのか、どのような意義があるのかを説明してください。
    4.  **関連用語**: この単語と一緒に覚えておくと良い関連キーワードを3つ挙げてください。

    以上の形式に従って、マークダウン形式で読みやすく回答を生成してください。
    """
    try:
        response = model.generate_content(prompt)
        return jsonify({"explanation": response.text})
    except Exception as e: return jsonify({"error": f"AIエラー: {e}"}), 500


# --- サーバーの起動 ---
if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f"✅ サーバーが起動しました！このURLをHTMLファイルで使います: {public_url}")
    app.run(port=5000)

✅ サーバーが起動しました！このURLをHTMLファイルで使います: NgrokTunnel: "https://4cb9d8d124cc.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Jul/2025 10:05:59] "OPTIONS /api/recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2025 10:06:08] "POST /api/recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2025 10:07:15] "OPTIONS /api/explain_term HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2025 10:07:26] "POST /api/explain_term HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2025 10:18:13] "OPTIONS /api/recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2025 10:18:18] "POST /api/recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2025 10:28:11] "OPTIONS /api/recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2025 10:28:12] "POST /api/recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Jul/2025 10:35:00] "OPTIONS /api/recommend